In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import random
import math
import time

# algorithm part

def euclidean_distance(city1, city2):
    x1, y1 = city1
    x2, y2 = city2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_tour_distance(tour, cities):
    total = 0
    for i in range(len(tour)):
        city1 = cities[tour[i]]
        city2 = cities[tour[(i + 1) % len(tour)]]  # Wrap around to start
        total += euclidean_distance(city1, city2)
    return total

def generate_random_cities(n, max_coord=1000):
    cities = []
    for i in range(n):
        x = random.uniform(0, max_coord)
        y = random.uniform(0, max_coord)
        cities.append((x, y))
    return cities

def two_opt_swap(tour, i, j):
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

def swap_cities(tour):
    new_tour = tour[:]
    i, j = random.sample(range(len(tour)), 2)
    new_tour[i], new_tour[j] = new_tour[j], new_tour[i]
    return new_tour

def simulated_annealing(cities, initial_temp=10000, cooling_schedule='exponential',
                       alpha=0.995, beta=10, max_iterations=10000, 
                       min_temp=0.01, neighborhood='2-opt'):
    n = len(cities)
    
    # Initialize with random tour
    current_tour = list(range(n))
    random.shuffle(current_tour)
    current_distance = calculate_tour_distance(current_tour, cities)
    
    # Track best solution
    best_tour = current_tour[:]
    best_distance = current_distance
    
    # Track history for visualization
    history = []
    
    # Simulated Annealing
    temperature = initial_temp
    iteration = 0
    
    while iteration < max_iterations and temperature > min_temp:
        # Generate neighbor
        if neighborhood == '2-opt':
            i, j = sorted(random.sample(range(n), 2))
            neighbor_tour = two_opt_swap(current_tour, i, j)
        else:  # swap
            neighbor_tour = swap_cities(current_tour)
        
        neighbor_distance = calculate_tour_distance(neighbor_tour, cities)
        
        # Calculate change in distance
        delta = neighbor_distance - current_distance
        
        # Accept or reject neighbor
        if delta < 0:  # Better solution
            current_tour = neighbor_tour
            current_distance = neighbor_distance
            
            # Update best
            if current_distance < best_distance:
                best_tour = current_tour[:]
                best_distance = current_distance
        else:  # Worse solution - accept with probability
            probability = math.exp(-delta / temperature)
            if random.random() < probability:
                current_tour = neighbor_tour
                current_distance = neighbor_distance
        
        # Record history (sample every 100 iterations)
        if iteration % 100 == 0:
            history.append({
                'iteration': iteration,
                'temperature': temperature,
                'current_distance': current_distance,
                'best_distance': best_distance
            })
        
        # Cool down
        if cooling_schedule == 'exponential':
            temperature *= alpha
        else:  # linear
            temperature -= beta
        
        iteration += 1
    
    return best_tour, best_distance, history

# Gui part

class TSPSolverGUI:
    def __init__(self, root):
        self.root = root
        root.title("TSP Solver - Simulated Annealing")
        root.geometry("1000x700")
        root.configure(bg="#f5f5f5")
        
        self.cities = []
        self.best_tour = None
        self.best_distance = None
        
        self.setup_gui()
    
    def setup_gui(self):
        """Create GUI components"""
        # Title
        title_frame = tk.Frame(self.root, bg="#f5f5f5")
        title_frame.pack(pady=10)
        
        tk.Label(title_frame, text="Traveling Salesperson Problem Solver",
                font=("Arial", 18, "bold"), bg="#f5f5f5", fg="#2c3e50").pack()
        tk.Label(title_frame, text="Using Simulated Annealing Algorithm",
                font=("Arial", 11), bg="#f5f5f5", fg="#7f8c8d").pack()
        
        # Main container
        main_frame = tk.Frame(self.root, bg="#f5f5f5")
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Left panel - Controls
        self.create_control_panel(main_frame)
        
        # Right panel - Visualization
        self.create_visualization_panel(main_frame)
    
    def create_control_panel(self, parent):
        """Create control panel with parameters"""
        control_frame = tk.Frame(parent, bg="white", relief=tk.RIDGE, bd=2)
        control_frame.pack(side=tk.LEFT, fill=tk.BOTH, padx=(0, 5), pady=5)
        
        # Section 1: City Generation
        tk.Label(control_frame, text="📍 City Generation", 
                font=("Arial", 12, "bold"), bg="white", fg="#2980b9").pack(anchor=tk.W, padx=10, pady=(10,5))
        
        cities_frame = tk.Frame(control_frame, bg="white")
        cities_frame.pack(fill=tk.X, padx=10, pady=5)
        
        tk.Label(cities_frame, text="Number of Cities:", bg="white").pack(side=tk.LEFT)
        self.n_cities = tk.Spinbox(cities_frame, from_=5, to=100, width=8, font=("Arial", 10))
        self.n_cities.delete(0, tk.END)
        self.n_cities.insert(0, "20")
        self.n_cities.pack(side=tk.LEFT, padx=5)
        
        tk.Button(control_frame, text="Generate Cities", command=self.generate_cities,
                 bg="#27ae60", fg="white", font=("Arial", 10, "bold"), 
                 padx=15, pady=5).pack(fill=tk.X, padx=10, pady=5)
        
        # Section 2: Algorithm Parameters
        tk.Label(control_frame, text="Algorithm Parameters", 
                font=("Arial", 12, "bold"), bg="white", fg="#2980b9").pack(anchor=tk.W, padx=10, pady=(15,5))
        
        # Initial Temperature
        param_frame1 = tk.Frame(control_frame, bg="white")
        param_frame1.pack(fill=tk.X, padx=10, pady=3)
        tk.Label(param_frame1, text="Initial Temp:", bg="white", width=15, anchor=tk.W).pack(side=tk.LEFT)
        self.initial_temp = tk.Entry(param_frame1, width=12)
        self.initial_temp.insert(0, "10000")
        self.initial_temp.pack(side=tk.LEFT, padx=5)
        
        # Cooling Schedule
        param_frame2 = tk.Frame(control_frame, bg="white")
        param_frame2.pack(fill=tk.X, padx=10, pady=3)
        tk.Label(param_frame2, text="Cooling:", bg="white", width=15, anchor=tk.W).pack(side=tk.LEFT)
        self.cooling_var = tk.StringVar(value="exponential")
        tk.Radiobutton(param_frame2, text="Exponential", variable=self.cooling_var, 
                      value="exponential", bg="white").pack(side=tk.LEFT)
        tk.Radiobutton(param_frame2, text="Linear", variable=self.cooling_var, 
                      value="linear", bg="white").pack(side=tk.LEFT)
        
        # Alpha (for exponential)
        param_frame3 = tk.Frame(control_frame, bg="white")
        param_frame3.pack(fill=tk.X, padx=10, pady=3)
        tk.Label(param_frame3, text="Alpha (exp):", bg="white", width=15, anchor=tk.W).pack(side=tk.LEFT)
        self.alpha = tk.Entry(param_frame3, width=12)
        self.alpha.insert(0, "0.995")
        self.alpha.pack(side=tk.LEFT, padx=5)
        
        # Beta (for linear)
        param_frame4 = tk.Frame(control_frame, bg="white")
        param_frame4.pack(fill=tk.X, padx=10, pady=3)
        tk.Label(param_frame4, text="Beta (linear):", bg="white", width=15, anchor=tk.W).pack(side=tk.LEFT)
        self.beta = tk.Entry(param_frame4, width=12)
        self.beta.insert(0, "10")
        self.beta.pack(side=tk.LEFT, padx=5)
        
        # Max Iterations
        param_frame5 = tk.Frame(control_frame, bg="white")
        param_frame5.pack(fill=tk.X, padx=10, pady=3)
        tk.Label(param_frame5, text="Max Iterations:", bg="white", width=15, anchor=tk.W).pack(side=tk.LEFT)
        self.max_iter = tk.Entry(param_frame5, width=12)
        self.max_iter.insert(0, "10000")
        self.max_iter.pack(side=tk.LEFT, padx=5)
        
        # Min Temperature
        param_frame6 = tk.Frame(control_frame, bg="white")
        param_frame6.pack(fill=tk.X, padx=10, pady=3)
        tk.Label(param_frame6, text="Min Temp:", bg="white", width=15, anchor=tk.W).pack(side=tk.LEFT)
        self.min_temp = tk.Entry(param_frame6, width=12)
        self.min_temp.insert(0, "0.01")
        self.min_temp.pack(side=tk.LEFT, padx=5)
        
        # Neighborhood
        param_frame7 = tk.Frame(control_frame, bg="white")
        param_frame7.pack(fill=tk.X, padx=10, pady=3)
        tk.Label(param_frame7, text="Neighborhood:", bg="white", width=15, anchor=tk.W).pack(side=tk.LEFT)
        self.neighborhood_var = tk.StringVar(value="2-opt")
        tk.Radiobutton(param_frame7, text="2-opt", variable=self.neighborhood_var, 
                      value="2-opt", bg="white").pack(side=tk.LEFT)
        tk.Radiobutton(param_frame7, text="Swap", variable=self.neighborhood_var, 
                      value="swap", bg="white").pack(side=tk.LEFT)
        
        # Solve button
        tk.Button(control_frame, text="SOLVE TSP", command=self.solve_tsp,
                 bg="#e74c3c", fg="white", font=("Arial", 13, "bold"), 
                 padx=20, pady=10).pack(fill=tk.X, padx=10, pady=15)
        
        # Results
        tk.Label(control_frame, text="Results", 
                font=("Arial", 12, "bold"), bg="white", fg="#2980b9").pack(anchor=tk.W, padx=10, pady=(5,5))
        
        self.result_text = scrolledtext.ScrolledText(control_frame, height=12, 
                                                     font=("Courier", 9), bg="#ecf0f1",
                                                     relief=tk.FLAT, padx=5, pady=5)
        self.result_text.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0,10))
        self.result_text.config(state=tk.DISABLED)
    
    def create_visualization_panel(self, parent):
        """Create visualization panel"""
        viz_frame = tk.Frame(parent, bg="white", relief=tk.RIDGE, bd=2)
        viz_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=(5, 0), pady=5)
        
        tk.Label(viz_frame, text="🗺️ Tour Visualization", 
                font=("Arial", 13, "bold"), bg="white", fg="#2980b9").pack(pady=10)
        
        # Canvas for drawing
        self.canvas = tk.Canvas(viz_frame, bg="#ecf0f1", width=600, height=550)
        self.canvas.pack(padx=10, pady=(0,10), fill=tk.BOTH, expand=True)
        
        # Info
        info_frame = tk.Frame(viz_frame, bg="#d5dbdb", relief=tk.RIDGE, bd=1)
        info_frame.pack(fill=tk.X, padx=10, pady=(0,10))
        
        tk.Label(info_frame, text="💡 How it works:",
                font=("Arial", 9, "bold"), bg="#d5dbdb").pack(anchor=tk.W, padx=8, pady=(5,2))
        tk.Label(info_frame, text="Simulated Annealing explores solutions by accepting worse\n"
                                  "solutions with decreasing probability as temperature cools,\n"
                                  "escaping local optima to find better global solutions.",
                font=("Arial", 8), bg="#d5dbdb", justify=tk.LEFT).pack(anchor=tk.W, padx=8, pady=(0,5))
    
    def generate_cities(self):
        """Generate random cities"""
        try:
            n = int(self.n_cities.get())
            if n < 3:
                messagebox.showerror("Error", "Need at least 3 cities!")
                return
            
            self.cities = generate_random_cities(n)
            self.best_tour = None
            self.best_distance = None
            
            self.draw_cities()
            self.update_results(f"Generated {n} random cities.\nReady to solve!")
            
        except ValueError:
            messagebox.showerror("Error", "Enter valid number!")
    
    def solve_tsp(self):
        """Solve TSP using Simulated Annealing"""
        if not self.cities:
            messagebox.showwarning("No Cities", "Generate cities first!")
            return
        
        try:
            # Get parameters
            initial_temp = float(self.initial_temp.get())
            cooling = self.cooling_var.get()
            alpha = float(self.alpha.get())
            beta = float(self.beta.get())
            max_iterations = int(self.max_iter.get())
            min_temp = float(self.min_temp.get())
            neighborhood = self.neighborhood_var.get()
            
            self.update_results("Solving... Please wait...")
            self.root.update()
            
            # Run algorithm
            start_time = time.time()
            self.best_tour, self.best_distance, history = simulated_annealing(
                self.cities,
                initial_temp=initial_temp,
                cooling_schedule=cooling,
                alpha=alpha,
                beta=beta,
                max_iterations=max_iterations,
                min_temp=min_temp,
                neighborhood=neighborhood
            )
            elapsed_time = time.time() - start_time
            
            # Display results
            result = f"Solution Found!\n\n"
            result += f"Cities: {len(self.cities)}\n"
            result += f"Best Distance: {self.best_distance:.2f}\n"
            result += f"Time: {elapsed_time:.2f} seconds\n"
            result += f"Iterations: {history[-1]['iteration'] if history else 0}\n\n"
            result += f"Parameters:\n"
            result += f"  Cooling: {cooling}\n"
            result += f"  Initial Temp: {initial_temp}\n"
            result += f"  Neighborhood: {neighborhood}\n\n"
            result += f"Tour: {' → '.join(map(str, self.best_tour[:10]))}{'...' if len(self.best_tour) > 10 else ''}"
            
            self.update_results(result)
            self.draw_solution()
            
            messagebox.showinfo("Success!", f"Best Distance: {self.best_distance:.2f}")
            
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid parameters!\n{e}")
    
    def update_results(self, text):
        """Update results text"""
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(1.0, text)
        self.result_text.config(state=tk.DISABLED)
    
    def draw_cities(self):
        """Draw cities on canvas"""
        self.canvas.delete("all")
        
        if not self.cities:
            return
        
        # Get canvas size
        canvas_width = self.canvas.winfo_width() or 600
        canvas_height = self.canvas.winfo_height() or 550
        padding = 30
        
        # Scale cities to fit canvas
        xs = [c[0] for c in self.cities]
        ys = [c[1] for c in self.cities]
        
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)
        
        def scale_x(x):
            return padding + (x - x_min) / (x_max - x_min) * (canvas_width - 2*padding)
        
        def scale_y(y):
            return padding + (y - y_min) / (y_max - y_min) * (canvas_height - 2*padding)
        
        # Draw cities
        for i, (x, y) in enumerate(self.cities):
            sx, sy = scale_x(x), scale_y(y)
            self.canvas.create_oval(sx-5, sy-5, sx+5, sy+5, 
                                   fill="#3498db", outline="#2980b9", width=2)
            self.canvas.create_text(sx, sy-15, text=str(i), 
                                   font=("Arial", 8, "bold"), fill="#2c3e50")
        
        # Mark depot (city 0) differently
        sx, sy = scale_x(self.cities[0][0]), scale_y(self.cities[0][1])
        self.canvas.create_oval(sx-7, sy-7, sx+7, sy+7, 
                               fill="#e74c3c", outline="#c0392b", width=2)
        self.canvas.create_text(sx, sy-18, text="DEPOT", 
                               font=("Arial", 8, "bold"), fill="#e74c3c")
    
    def draw_solution(self):
        """Draw the solution tour"""
        self.draw_cities()
        
        if not self.best_tour:
            return
        
        # Get canvas size
        canvas_width = self.canvas.winfo_width() or 600
        canvas_height = self.canvas.winfo_height() or 550
        padding = 30
        
        # Scale cities
        xs = [c[0] for c in self.cities]
        ys = [c[1] for c in self.cities]
        
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)
        
        def scale_x(x):
            return padding + (x - x_min) / (x_max - x_min) * (canvas_width - 2*padding)
        
        def scale_y(y):
            return padding + (y - y_min) / (y_max - y_min) * (canvas_height - 2*padding)
        
        # Draw tour
        for i in range(len(self.best_tour)):
            city1_idx = self.best_tour[i]
            city2_idx = self.best_tour[(i + 1) % len(self.best_tour)]
            
            x1, y1 = self.cities[city1_idx]
            x2, y2 = self.cities[city2_idx]
            
            sx1, sy1 = scale_x(x1), scale_y(y1)
            sx2, sy2 = scale_x(x2), scale_y(y2)
            
            self.canvas.create_line(sx1, sy1, sx2, sy2, 
                                   fill="#27ae60", width=2, arrow=tk.LAST)


# MAIN PROGRAM

def main():
    root = tk.Tk()
    app = TSPSolverGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()